In [4]:
import torch
import torchvision
from IPython.core.display import display, HTML
from PIL import Image, ImageDraw, ImageFont
import base64
import io
import pandas as pd
import os
import json
import re
import sys

/Users/hyunsubong/opt/anaconda3/envs/hyun_ml/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/hyunsubong/opt/anaconda3/envs/hyun_ml/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  Referenced from: <CAE66874-17C2-35C9-9C4D-6BA9770AF17F> /Users/hyunsubong/opt/anaconda3/envs/hyun_ml/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <7AB234BF-29E3-390D-8C8D-C6920F94393C> /Users/hyunsubong/opt/anaconda3/envs/hyun_ml/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")
/var/folders/97/h6gs8h1d6k14yqbrgsk0nrw80000gn/T/ipykernel_84582/534817885.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please imp

# CRAFT

### Ubuntu 22.04 LTS CUDA

In [ ]:
craft = !CUDA_VISIBLE_DEVICES=0 python3 craft/craft_test.py \
            --trained_model="./weights/craft/craft_mlt_25k.pth" \
            --refiner_model="./weights/craft/craft_refiner_CTW1500.pth" \
            --test_folder="./data/test_img/IMG_1257/" \
            --cuda=True \
            --mps=False

### Apple Silicon Mps

In [19]:
craft = !python craft/craft_test.py \
            --trained_model="./weights/craft/craft_mlt_25k.pth" \
            --refiner_model="./weights/craft/craft_refiner_CTW1500.pth" \
            --test_folder="./data/test_img/IMG_1331/" \
            --cuda=False \
            --mps=True

# Dialogue crop

In [20]:
craft

['/Users/hyunsubong/opt/anaconda3/envs/hyun_ml/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/hyunsubong/opt/anaconda3/envs/hyun_ml/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE',
 '  Referenced from: <CAE66874-17C2-35C9-9C4D-6BA9770AF17F> /Users/hyunsubong/opt/anaconda3/envs/hyun_ml/lib/python3.9/site-packages/torchvision/image.so',
 '  Expected in:     <7AB234BF-29E3-390D-8C8D-C6920F94393C> /Users/hyunsubong/opt/anaconda3/envs/hyun_ml/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib',
 '  warn(f"Failed to load image Python extension: {e}")',
 '/Users/hyunsubong/opt/anaconda3/envs/hyun_ml/lib/python3.9/site-packages/torchvision/models/_utils.py:252: UserWarning: Accessing the model URLs via the internal dictionar

In [21]:
input_img_dir = './data/test_img/IMG_1331/'
image_bbox_dir = input_img_dir + 'result/'
image_path = ''
filename = ''

for idx in range(len(craft)):
    try:
        ind = craft[idx].index(input_img_dir)
        filename = craft[idx][ind+len(input_img_dir):]
        print(filename)
        image_path = input_img_dir + filename
    except:
        continue
        
image_bbox_path = f"{image_bbox_dir}res_{filename[:-4]}.txt"
image_bbox = open(image_bbox_path, 'r')
image_bbox = image_bbox.read().splitlines()

image_bbox[0]

IMG_1331.PNG


'151,63,279,63,279,115,151,115'

In [22]:
print(image_bbox[0].split(','))

['151', '63', '279', '63', '279', '115', '151', '115']


In [23]:
image_bbox_int = []
for idx in range(len(image_bbox)):
    temp = image_bbox[idx].split(',')
    image_bbox_int.append(list(map(int, temp)))

## Boundingbox 정렬

In [24]:
image1 = Image.open(image_path)

In [25]:
def get_box_precedence(box, cols):
    tolerance_factor = 30
    return ((box[1] // tolerance_factor) * tolerance_factor) * cols + box[0]

def get_sorted_bb(boxes, img_shape):
    sorted_boxes = sorted(boxes, key=lambda x: get_box_precedence(x, img_shape[1]))
    return sorted_boxes

In [26]:
image_bbox_int = get_sorted_bb(image_bbox_int, image1.size)
# sorted_boxes

In [27]:
image1 = Image.open(image_path)
for idx, bboxs in enumerate(image_bbox_int):
    print(bboxs)
    x = bboxs[0]
    y = bboxs[1]
    x2 = bboxs[4]
    y2 = bboxs[5]
    
    # crop(죄측 상단 x,y , 우측 하단 x,y)
    croppedImage = image1.crop((x, y, x2, y2))
    
    if not os.path.isdir(f"{input_img_dir}crop"):
        os.mkdir(f"{input_img_dir}crop")
    croppedImage.save(f"{input_img_dir}crop/crop_{filename[:-4]}_{idx}.PNG")

# [974, 615, 1130, 615, 1130, 666, 974, 666] # 오
#  [774, 623, 806, 623, 806, 658, 774, 658] # 좋지 좋지
# y 차이가 8이하면 같은 줄이라 판단

#[391, 962, 415, 962, 415, 986, 391, 986],
# [571, 946, 666, 946, 666, 990, 571, 990])
# y 차이 16

[151, 63, 279, 63, 279, 115, 151, 115]
[922, 71, 978, 71, 978, 107, 922, 107]
[95, 203, 179, 203, 179, 247, 95, 247]
[523, 199, 654, 199, 654, 255, 523, 255]
[152, 326, 249, 332, 246, 373, 149, 367]
[183, 415, 383, 415, 383, 467, 183, 467]
[387, 419, 519, 419, 519, 467, 387, 467]
[523, 419, 635, 419, 635, 459, 523, 459]
[642, 415, 762, 415, 762, 471, 642, 471]
[766, 415, 846, 415, 846, 467, 766, 467]
[850, 419, 934, 419, 934, 467, 850, 467]
[183, 471, 263, 471, 263, 523, 183, 523]
[267, 471, 395, 471, 395, 523, 267, 523]
[974, 511, 1050, 511, 1050, 547, 974, 547]
[1050, 515, 1098, 515, 1098, 547, 1050, 547]
[806, 623, 974, 623, 974, 666, 806, 666]
[978, 623, 1138, 623, 1138, 674, 978, 674]
[642, 666, 714, 666, 714, 694, 642, 694]
[718, 662, 766, 662, 766, 694, 718, 694]
[694, 746, 774, 746, 774, 786, 694, 786]
[781, 746, 977, 740, 979, 794, 783, 801]
[986, 754, 1126, 754, 1126, 782, 986, 782]
[535, 790, 611, 790, 611, 818, 535, 818]
[619, 790, 662, 790, 662, 818, 619, 818]
[152, 861, 2

# TPS-ResNet

### Ubuntu 22.04 LTS CUDA

In [ ]:
recognition = !CUDA_VISIBLE_DEVICES=0 python3 tps-resnet/demo.py \
                        --Transformation TPS --FeatureExtraction VGG --SequenceModeling BiLSTM --Prediction Attn \
                        --image_folder="data/test_img/IMG_1257/crop/" \
                        --saved_model="weights/TPS-ResNet/pretrained/best.pth"

### Apple Silicon Mps

In [ ]:
# output = !python tps-resnet/demo.py \
#                     --Transformation TPS --FeatureExtraction VGG --SequenceModeling BiLSTM --Prediction Attn \
#                     --image_folder="data/test_img/IMG_1257/crop/" \
#                     --saved_model="weights/TPS-ResNet/pretrained/best.pth"
!python tps-resnet/demo.py \
                    --Transformation TPS --FeatureExtraction VGG --SequenceModeling BiLSTM --Prediction Attn \
                    --image_folder="data/test_img/IMG_1257/crop/" \
                    --saved_model="weights/TPS-ResNet/pretrained/best.pth"

# Dialogue Summarize

In [5]:
with open('json_sample.json', encoding='utf-8') as data:
        json_data = json.load(data)

In [6]:
images_ls = json_data.get('images')
fields_ls = images_ls[0]['fields']

In [7]:
bbox_ls = []
for idx, field in enumerate(fields_ls):
    bbox_temp = []
    inferText = field['inferText']
    boundingPoly = field['boundingPoly']['vertices']
    
    # 왼쪽 위, 오른쪽 위, 오른쪽 아래,왼쪽 아래 로 저장
    for vertice in boundingPoly[0:]:
        x = vertice['x']
        y = vertice['y']
        bbox_temp.append(int(x))
        bbox_temp.append(int(y))
    
    bbox_temp.append(inferText)
    
    bbox_ls.append(bbox_temp)

## 한글인지 확인

##### 처음 나오는 한글이라면 대화 상대이므로 이 대화 상대가 대화 시간 이후로 처음 나오는 경우에는 제거대상

In [8]:
def isHangul(text):
    #Check the Python Version
    pyVer3 =  sys.version_info >= (3, 0)

    if pyVer3 : # for Ver 3 or later
        encText = text
    else: # for Ver 2.x
        if type(text) is not unicode:
            encText = text.decode('utf-8')
        else:
            encText = text

    hanCount = len(re.findall(u'[\u3130-\u318F\uAC00-\uD7A3]+', encText))
    return hanCount > 0

## 대화만 추출

In [9]:
participant = ''
for idx, row in enumerate(bbox_ls):
    if isHangul(row[-1]) == True:
        participant = row[-1]
        bbox_ls = bbox_ls[idx+1:]
        break

new_bbox_ls = []
dialogue = []
for idx, row in enumerate(bbox_ls):
    if row[-1] == participant:
        continue
    elif row[-1] == 'PM' or row[-1] == 'AM':
        new_bbox_ls.append(dialogue)
        dialogue = []
        continue
    elif row[-1] in ':':
        continue
    
    if isHangul(row[-1]):
        dialogue.append(row[-1])

In [10]:
new_bbox_ls

[['얌', '도은이랑', '5/21에', '보기로', '했는데너도', '시간', '되면', '올랴??'],
 ['그래??', '좋지좋지'],
 ['되게만들어야지'],
 ['크', '오키오키!'],
 ['아직', '어디서', '볼지는', '안', '정한거지?'],
 ['웅웅', '날짜밖에', '안정했어', '크', '크', '크'],
 ['부발역은'],
 ['어디가', '좋은데??'],
 ['안되나요', '크']]